In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('logreg').getOrCreate()

In [3]:
df = spark.sql('select * from customer_churn')

In [4]:
df.show(5)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
Cameron Williams|42.0| 11066.8| 0| 7.22| 8.0|2013-08-30 07:00:40|10265 Elizabeth M...| Harvey LLC| 1|
 Kevin Mueller|41.0| 11916.22| 0| 6.5| 11.0|2013-08-13 00:38:46|6157 Frank Garden...| Wilson PLC| 1|
 Eric Lozano|38.0| 12884.75| 0| 6.67| 12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...| 1|
 Phillip White|42.0| 8010.76| 0| 6.71| 10.0|2014-04-22 12:43:12|13120 Daniel Moun...| Smith Inc| 1|
 Cynthia Norton|37.0| 9191.58| 0| 5.56| 9.0|2016-01-19 15:31:15|765 Tricia Row Ka...| Love-Jones| 1|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
only showing top 5 rows

In [5]:
df.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- Churn: integer (nullable = true)

In [6]:
df2 = df.select(['Age','Total_Purchase','Years','Num_Sites','Churn'])

In [7]:
df2.groupBy('Churn').count().show()

+-----+-----+
Churn|count|
+-----+-----+
 1| 150|
 0| 750|
+-----+-----+

In [8]:
df2.show(5)

+----+--------------+-----+---------+-----+
 Age|Total_Purchase|Years|Num_Sites|Churn|
+----+--------------+-----+---------+-----+
42.0| 11066.8| 7.22| 8.0| 1|
41.0| 11916.22| 6.5| 11.0| 1|
38.0| 12884.75| 6.67| 12.0| 1|
42.0| 8010.76| 6.71| 10.0| 1|
37.0| 9191.58| 5.56| 9.0| 1|
+----+--------------+-----+---------+-----+
only showing top 5 rows

In [9]:
from pyspark.ml.feature import VectorAssembler

In [10]:
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Years', 'Num_Sites'],outputCol='features')
output = assembler.transform(df2)

In [11]:
output.show(5)

+----+--------------+-----+---------+-----+--------------------+
 Age|Total_Purchase|Years|Num_Sites|Churn| features|
+----+--------------+-----+---------+-----+--------------------+
42.0| 11066.8| 7.22| 8.0| 1|[42.0,11066.8,7.2...|
41.0| 11916.22| 6.5| 11.0| 1|[41.0,11916.22,6....|
38.0| 12884.75| 6.67| 12.0| 1|[38.0,12884.75,6....|
42.0| 8010.76| 6.71| 10.0| 1|[42.0,8010.76,6.7...|
37.0| 9191.58| 5.56| 9.0| 1|[37.0,9191.58,5.5...|
+----+--------------+-----+---------+-----+--------------------+
only showing top 5 rows

In [12]:
final_data = output.select('features','Churn')

In [13]:
train,test = final_data.randomSplit([0.7,0.3])

In [14]:
from pyspark.ml.classification import LogisticRegression

In [15]:
log = LogisticRegression(featuresCol='features',labelCol='Churn',predictionCol='prediction')

In [16]:
log_model = log.fit(train)

In [17]:
log_model_summary =log_model.summary

In [18]:
print(log_model_summary.falsePositiveRateByLabel)
print(log_model_summary.recallByLabel)
print(log_model_summary.areaUnderROC)
print(log_model_summary.accuracy)
print(log_model_summary.precisionByLabel)

[0.5425531914893617, 0.03153988868274583]
[0.9684601113172542, 0.4574468085106383]
0.894761773181
0.892575039494
[0.9109947643979057, 0.7166666666666667]

In [19]:
pred = log_model.evaluate(test)

In [20]:
print(pred.falsePositiveRateByLabel)
print(pred.recallByLabel)
print(pred.areaUnderROC)
print(pred.accuracy)
print(pred.precisionByLabel)
print(pred.predictions.show())

[0.35714285714285715, 0.03317535545023697]
[0.966824644549763, 0.6428571428571429]
0.924001354096
0.898876404494
[0.9107142857142857, 0.8372093023255814]
+--------------------+-----+--------------------+--------------------+----------+
 features|Churn| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[25.0,9672.03,5.4...| 0|[4.05788423467927...|[0.98300816070598...| 0.0|
[28.0,11128.95,5....| 0|[4.05575057964775...|[0.98297248526441...| 0.0|
[29.0,8688.17,5.7...| 1|[2.70401011048935...|[0.93726285710374...| 0.0|
[29.0,9617.59,5.4...| 0|[3.86788599336289...|[0.97952545852647...| 0.0|
[30.0,7960.64,2.7...| 1|[3.31020866721503...|[0.96477737240989...| 0.0|
[30.0,13473.35,3....| 0|[2.40473794971478...|[0.91718788307152...| 0.0|
[31.0,5304.6,5.29...| 0|[3.00449714352075...|[0.95277687974216...| 0.0|
[31.0,8688.21,3.5...| 0|[5.99848505881220...|[0.99752363740906...| 0.0|
[31.0,8829.83,4.5...| 0|[4.36508195416804...|[0.98744599358272...| 0.0|
[31.0,11743.24,5....| 0|[5.95478249134372...|[0.99741330050488...| 0.0|
[32.0,5756.12,5.9...| 0|[3.67322000906772...|[0.97523434566205...| 0.0|
[32.0,8011.38,5.3...| 0|[1.70447980344359...|[0.84611891774172...| 0.0|
[32.0,9036.27,7.1...| 0|[-0.4582288244810...|[0.38740607993725...| 1.0|
[32.0,11540.86,3....| 0|[6.14621724604099...|[0.99786300902025...| 0.0|
[32.0,13630.93,4....| 0|[1.99147658253619...|[0.87989926522979...| 0.0|
[33.0,12638.51,5....| 0|[3.80581658447476...|[0.97824287280611...| 0.0|
[34.0,6461.86,4.6...| 0|[4.28233410047185...|[0.98637773912923...| 0.0|
[34.0,9265.59,5.7...| 0|[3.48353012605058...|[0.97021550110521...| 0.0|
[34.0,10674.92,3....| 0|[3.86375915915828...|[0.97944252949315...| 0.0|
[34.0,12397.49,4....| 0|[2.72703090771234...|[0.93860295863111...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

None

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [22]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Churn')

In [23]:
auc = evaluator.evaluate(pred.predictions)
auc

Out[ 64 ]: 0.8048408937034529

In [24]:
# prediction on new data
new_data = spark.sql('select * from new_customers')

In [25]:
# fitting data on whole data(previous data)
log = LogisticRegression(labelCol='Churn')
final_model = log.fit(final_data)

In [26]:
# assembler is taking the selected variables from new_data as defined by assembler object, not all the varibales from new_data
new_data2 = assembler.transform(new_data) 

In [27]:
final_results = final_model.transform(new_data2)

In [28]:
final_results.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company| features| rawPrediction| probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
 Andrew Mccall|37.0| 9935.53| 1| 7.71| 8.0|2011-08-29 18:37:54|38612 Johnny Stra...| King Ltd|[37.0,9935.53,7.7...|[2.41832366013597...|[0.91821394450559...| 0.0|
Michele Wright|23.0| 7526.94| 1| 9.28| 15.0|2013-07-22 18:19:54|21083 Nicole Junc...| Cannon-Benson|[23.0,7526.94,9.2...|[-5.9906678174824...|[0.00249574827398...| 1.0|
 Jeremy Chang|65.0| 100.0| 1| 1.0| 15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|[65.0,100.0,1.0,1...|[-3.4325297516674...|[0.03129415226295...| 1.0|
Megan Ferguson|32.0| 6487.5| 0| 9.4| 14.0|2016-10-28 05:32:13|922 Wright Branch...| Sexton-Golden|[32.0,6487.5,9.4,...|[-5.3429693908695...|[0.00475889563977...| 1.0|
 Taylor Young|32.0| 13147.71| 1| 10.0| 8.0|2012-03-20 00:36:46|Unit 0789 Box 073...| Wood LLC|[32.0,13147.71,10...|[1.28437178442218...|[0.78319303188952...| 0.0|
 Jessica Drake|22.0| 8445.26| 1| 3.46| 14.0|2011-02-04 19:29:27|1148 Tina Straven...| Parks-Robbins|[22.0,8445.26,3.4...|[-1.4385210975368...|[0.19177446955641...| 1.0|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+